In [ ]:
%run -i 'classes/libraries.py'

In [ ]:
# Import all the necessary libraries used inside the system
from classes import ethData, models 

In [ ]:
m = models.models()

In [ ]:
df_nasdaq = m.getNasdaqData()

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
# ---------------------------------------------------
# |_______________________________|_________|________|
#           TRAIN                    TEST      PRED
#                                         today

from_data = (datetime.now() - relativedelta(days=60) - relativedelta(years=4)).strftime('%Y-%m-%d')
to_data = (datetime.now() - relativedelta(days=60)).strftime('%Y-%m-%d')
today = (datetime.now()).strftime('%Y-%m-%d')

In [ ]:
models = {}
rmse = {}
for symbol in df_nasdaq['symbol']:
    try:
        df = m.getStockQuote(web, symbol, 'yahoo', from_data, today)
        if(df.notnull and df.shape[0] >= 60):
            print("Ticker Analyzed: {0}".format(symbol))
            print(df.shape)
            df_filtered = df.filter(['Close'])
            perc_train = .8
            models[symbol], rmse[symbol] = m.modelBase(df_filtered, 60, perc_train, 10, True)
            models[symbol].save('models/model_nasdaq_{0}'.format(symbol))
            df_test = df_filtered.tail(math.ceil(df_filtered.shape[0] * 1 - perc_train))
            print("Accuracy: {0}".format(rmse[symbol] / (df_test.max() - df_test.min())))
    except:
        print("Error on {0}!".format(symbol))

In [ ]:
from_data

In [ ]:
to_data

In [ ]:
today

In [ ]:
df_nasdaq['symbol'][10]

In [ ]:
df_filtered = df.filter(['Close'])

In [ ]:
df_filtered.tail(math.ceil(df_filtered.shape[0] * 0.2)).max()

In [ ]:
models = {}
rmse = {}
perc_train = .8
symbol = df_nasdaq['symbol'][10]
df_test = df_filtered.tail(math.ceil(df_filtered.shape[0] * 1 - perc_train))
models[symbol], rmse[symbol] = m.modelBase(df_filtered, 60, perc_train, 3, False)
print("Accuracy: {0}".format(rmse[symbol] / (df_test.max() - df_test.min())))

In [ ]:
prev_data = m.getStockQuote(web, df_nasdaq['symbol'][10], 'yahoo', to_data, today).values
scaler = MinMaxScaler(feature_range=(0,1))
prev_data_scaled = scaler.fit_transform(prev_data)
X_test = []
X_test.append(prev_data_scaled)
X_test = np.array(X_test)
X_test

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
pred_price = models[symbol].predict(X_test)
pred_price = scaler.invcerse_transform(pred_price)

In [ ]:
import pandas as pd
df = pd.read_csv('data/cryptoStock/ethbtc.csv')['close']

In [ ]:
import pandas as pd
crypto = 'ethbtc'
df = pd.read_csv('data/cryptoStock/{0}.csv'.format(crypto))['close']
spectrumRatio = 1
df = df.groupby(df.index // spectrumRatio).mean().to_frame()
df_filtered = df
from classes import ethData, models
m = models.models()
model = m.modelBase(df_filtered, 60, .8, 10, True)
model.save('models/model_crypto_{0}'.format(crypto))

In [ ]:
#df[df.columns[0]].to_frame()

In [ ]:
#df.rename(columns = {df.columns[0] : 'Data'})

In [ ]:
df_filtered = df
from classes import ethData, models
m = models.models()
m.modelBase(df_filtered, 60, .8, 10, True)

In [ ]:
import matplotlib.pyplot as plt
data = [1, 2, 3, 7, 5, 6, 7 , 8, 9]
predictions = [1, 2, 3, 4, 5, 7, 7 , 8, 9]


In [ ]:
valid = data[5:8]
valid

In [ ]:
valid.insert(1,'Predictions',predictions, True)
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Spectrum', fontsize=18)
plt.ylabel('Price', fontsize=18)
plt.plot(valid[[data.columns[0], 'Predictions']])
plt.legend(['Val', 'Predictions'], loc = 'lower right')
plt.show()

In [ ]:
df.shape

In [ ]:
models

In [ ]:

data = df.filter(['Close'])
dataset = data.values


training_data_len = math.ceil(len(dataset) * .8) #80/20 or 70/30

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
train_data = scaled_data[0:training_data_len, :]

#Creating training dataset

x_train = []
y_train = []

windowLenght = 60

for i in range(windowLenght, len(train_data)):
    x_train.append(train_data[i - windowLenght:i, 0])
    y_train.append(train_data[i, 0])

# Convert and reshape x_train and y_train to numpy

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

#Build the LSTM Model (50/50/25 are the number of neurons)

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

#Compile the model

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae', 'mape'])

#Train the model

model.fit(x_train, y_train, batch_size = 1, epochs = 10)

#Creating testing dataset

test_data = scaled_data[training_data_len - windowLenght: , :]

#Create dataset x_test and y_test

x_test = []
y_test = dataset[training_data_len: , :]

for i in range(windowLenght, len(test_data)):
    x_test.append(test_data[i - windowLenght:i, 0])

x_test = np.array(x_test)

x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

#Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)



In [ ]:
#Plot data
train = data[:training_data_len]
valid = data[training_data_len:]


In [ ]:
valid.insert(1,'Predictions',predictions, True)

In [ ]:
valid

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Prediction'], loc = 'lower right')
plt.show()

In [ ]:
df = m.getStockQuote(web, df_nasdaq['symbol'][10], 'yahoo', from_data, to_data)
new_df = df.filter(['Close'])
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
last_60_days = new_df[-60:].values
last_60_days_scaled = scaler.fit_transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
pred_price = models[symbol].predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
pred_price

In [ ]:
df = m.getStockQuote(web, df_nasdaq['symbol'][10], 'yahoo', to_data, today)
df

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Prediction'], loc = 'lower right')
plt.show()

In [ ]:
last_60_days_scaled = np.insert(last_60_days_scaled, 0, pred_price)


In [ ]:
plt.figure(figsize=(16,8))
plt.title('Close Price HIstory')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Current Price USD ($)', fontsize= 18)


In [ ]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8) #80/20

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
#Creating training dataset
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
windowLenght = 60
for i in range(windowLenght, len(train_data)):
    x_train.append(train_data[i - windowLenght:i, 0])
    y_train.append(train_data[i, 0])
    if i <= windowLenght:
        print(x_train)
        print(y_train)
        

In [ ]:
# Convert x_train and y_train to numpy
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the Data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#Build the LSTM Model (50/50/25 are the number of neurons)

model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model
model.fit(x_train, y_train, batch_size = 1, epochs = 100)

In [ ]:
#Creating testing dataset
test_data = scaled_data[training_data_len - windowLenght: , :]
#Create dataset x_test and y_test
x_test = []
y_test = dataset[training_data_len: , :]
for i in range(windowLenght, len(test_data)):
    x_test.append(test_data[i - windowLenght:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test) ** 2)
rmse

In [ ]:
#Plot data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.show()

In [1]:
from classes import utilities
util = utilities.utilities()
info = util.getFrameworkInfo()

ModuleNotFoundError: No module named 'classes'

In [ ]:
print(info)